# Create metadata

In [ ]:
import pandas as pd
import numpy as np

# Aridity calculations

In [ ]:
aridity = pd.read_csv("../../data/latlongs/test_plots_aridity.csv")
aridity['aiet0'] /= 10000
aridity['aridity'] = pd.cut(aridity['aiet0'],
                            [0, 0.03, 0.2, 0.5, 0.65, np.max(aridity['aiet0'])],
                            labels = ['Hyper Arid', 'Arid', 'Semi-Arid',
                            'Dry Subhumid', 'Humid'])

In [ ]:
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
continents = pd.read_csv("../../data/metrics/metadata/metadata.csv")
continents.head(5)

In [ ]:
continents = pd.read_csv("../../data/latlongs/test_continents.csv")
continents.head(5)

In [ ]:
clouds = []
slopes = []
ids = continents['id']
for i in ids:
    cloud = np.load("../../data/raw/test-clouds/{}.npy".format(str(i)))
    data = np.load("../../data/test-s2/{}.npy".format(str(i)))
    slopes.append(np.mean(data[0, :, :, 10]))
    cloud_percents = [len(np.argwhere(cloud[i].flatten() > 0.3)) for i in range(cloud.shape[0])]
    cloud_percents = np.array(cloud_percents) / cloud.shape[1]*cloud.shape[2]
    cloud_percents = len(np.argwhere(cloud_percents > 0.2)) / len(cloud_percents)
    clouds.append(cloud_percents)

In [ ]:
continents['clouds'] = clouds
continents['slopes'] = slopes
continents.to_csv("../../data/metrics/metadata/metadata.csv")

# Load proposed model predictions

In [ ]:
proposed = pd.read_csv("../../data/metrics/proposed-sample.csv")
proposed = proposed.join(continents, how = 'inner')
proposed.head(5)

In [ ]:
means_l = []
means = []
means_u = []
proposed['ccgroup'] = 0
proposed['ccgroup'][(proposed['group'] >= 30) & (proposed['group'] <= 70)] = 1
for i in range(0, 100, 10):
    group = proposed[proposed['group'] == i]
    ci = mean_confidence_interval(group['error'], 0.95)
    if i >= 30 and i <= 70:
        means_l.append(ci[0])
        means_u.append(ci[2])
        means.append(ci[1])
        print(i, ci)
print(np.mean(means_l), np.mean(means), np.mean(means_u))

print(mean_confidence_interval(proposed[proposed['ccgroup'] == 1]['error']))

In [ ]:
mean_confidence_interval(proposed['error'], 0.95)

# Overall metrics

In [ ]:
tp = proposed.sum()['tp_soft']
fp = proposed.sum()['fp_soft']
fn = proposed.sum()['fn_soft']
precision = tp / (tp + fp)
recall = tp / (tp + fn)
print(float(precision), float(recall))

In [ ]:
np.mean(proposed['error'])

# Aridity Metrics

In [ ]:
proposed = proposed.join(aridity)
proposed.head(5)

In [ ]:
sums = proposed.groupby("aridity").sum()
for i in proposed['aridity'].unique():
    tp = sums[sums.index == i]['tp_soft']
    fp = sums[sums.index == i]['fp_soft']
    fn = sums[sums.index == i]['fn_soft']
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print(i, float(precision), float(recall))

## Regional metrics

In [ ]:
sums = proposed.groupby("CONTINENT").sum()
for i in ['Africa', 'Asia', "Australia", "Europe", "North America", "South America"]:
    tp = sums[sums.index == i]['tp_soft']
    fp = sums[sums.index == i]['fp_soft']
    fn = sums[sums.index == i]['fn_soft']
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print(i, float(precision), float(recall))

# Cloud metrics

In [ ]:
tp = proposed[proposed['clouds'] < 0.75].sum()['tp_soft']
fp = proposed[proposed['clouds'] < 0.75].sum()['fp_soft']
fn = proposed[proposed['clouds'] < 0.75].sum()['fn_soft']
error = proposed[proposed['clouds'] < 0.75].mean()['error']
count = proposed[proposed['clouds'] < 0.75].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 0.75 cloud: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['clouds'] >= 0.75].sum()['tp_soft']
fp = proposed[proposed['clouds'] >= 0.75].sum()['fp_soft']
fn = proposed[proposed['clouds'] >= 0.75].sum()['fn_soft']
error = proposed[proposed['clouds'] >= 0.75].mean()['error']
count = proposed[proposed['clouds'] >= 0.75].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Above 0.75 cloud: ", precision, recall, error, count)

# Slope metrics

In [ ]:
tp = proposed[proposed['slopes'] < 0.1].sum()['tp_soft']
fp = proposed[proposed['slopes'] < 0.1].sum()['fp_soft']
fn = proposed[proposed['slopes'] < 0.1].sum()['fn_soft']
error = proposed[proposed['slopes'] < 0.1].mean()['error']
count = proposed[proposed['slopes'] < 0.1].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 0.1 slope: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['slopes'] >= 0.1].sum()['tp_soft']
fp = proposed[proposed['slopes'] >= 0.1].sum()['fp_soft']
fn = proposed[proposed['slopes'] >= 0.1].sum()['fn_soft']
error = proposed[proposed['slopes'] >= 0.1].mean()['error']
count = proposed[proposed['slopes'] >= 0.1].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Above 0.1 slope: ", precision, recall, error, count)

# Canopy metrics

In [ ]:
tp = proposed[proposed['true'] < 20].sum()['tp_soft']
fp = proposed[proposed['true'] < 20].sum()['fp_soft']
fn = proposed[proposed['true'] < 20].sum()['fn_soft']
error = proposed[proposed['true'] < 20].mean()['error']
count = proposed[proposed['true'] < 20].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 20%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'].between(20, 40)].sum()['tp_soft']
fp = proposed[proposed['true'].between(20, 40)].sum()['fp_soft']
fn = proposed[proposed['true'].between(20, 40)].sum()['fn_soft']

error = proposed[proposed['true'].between(20, 40)].mean()['error']
count = proposed[proposed['true'].between(20, 40)].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Between 20 and 40%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'].between(0, 40)].sum()['tp_soft']
fp = proposed[proposed['true'].between(0, 40)].sum()['fp_soft']
fn = proposed[proposed['true'].between(0, 40)].sum()['fn_soft']

error = proposed[proposed['true'].between(0, 40)].mean()['error']
count = proposed[proposed['true'].between(0, 40)].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Between 0 and 40%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'].between(40, 60)].sum()['tp_soft']
fp = proposed[proposed['true'].between(40, 60)].sum()['fp_soft']
fn = proposed[proposed['true'].between(40, 60)].sum()['fn_soft']

error = proposed[proposed['true'].between(40, 60)].mean()['error']
count = proposed[proposed['true'].between(40, 60)].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Between 40 and 60%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'].between(60, 101)].sum()['tp_soft']
fp = proposed[proposed['true'].between(60, 101)].sum()['fp_soft']
fn = proposed[proposed['true'].between(60, 101)].sum()['fn_soft']

error = proposed[proposed['true'].between(60, 101)].mean()['error']
count = proposed[proposed['true'].between(60, 101)].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Between 60 and 100%: ", precision, recall, error, count)

In [ ]:
proposed = pd.read_csv("../../data/metrics/proposed.csv")
print(proposed.mean()['soft_prec'], proposed.mean()['soft_rec'])

# Load Random forest predictions

In [ ]:
proposed = pd.read_csv("../../data/metrics/rf.csv")
print(proposed.mean()['soft_prec'], proposed.mean()['soft_rec'])

In [ ]:
proposed = pd.read_csv("../../data/metrics/rf-sample.csv")
proposed = proposed.drop(['Unnamed: 0'], axis = 1)
proposed = proposed.drop(['lats'], axis = 1)
proposed = proposed.drop(['longs'], axis = 1)


proposed = proposed.join(continents, how = 'inner')
proposed.head(5)

In [ ]:
proposed = proposed.join(aridity)
sums = proposed.groupby("aridity").sum()
for i in proposed['aridity'].unique():
    tp = sums[sums.index == i]['tp_soft']
    fp = sums[sums.index == i]['fp_soft']
    fn = sums[sums.index == i]['fn_soft']
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print(i, float(precision), float(recall))

In [ ]:
means_l = []
means = []
means_u = []
proposed['ccgroup'] = 0
proposed['ccgroup'][(proposed['group'] >= 30) & (proposed['group'] <= 70)] = 1
for i in range(0, 100, 10):
    group = proposed[proposed['group'] == i]
    ci = mean_confidence_interval(group['error'], 0.95)
    if i >= 30 and i <= 70:
        means_l.append(ci[0])
        means_u.append(ci[2])
        means.append(ci[1])
        print(i, ci)
print(np.mean(means_l), np.mean(means), np.mean(means_u))

print(mean_confidence_interval(proposed[proposed['ccgroup'] == 1]['error']))

In [ ]:
mean_confidence_interval(proposed['error'], 0.95)

In [ ]:
sums = proposed.groupby("CONTINENT").sum()
for i in ['Africa', 'Asia', "Australia", "Europe", "North America", "South America"]:
    tp = sums[sums.index == i]['tp_soft']
    fp = sums[sums.index == i]['fp_soft']
    fn = sums[sums.index == i]['fn_soft']
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print(i, float(precision), float(recall))

In [ ]:
tp = proposed[proposed['true'] < 20].sum()['tp_soft']
fp = proposed[proposed['true'] < 20].sum()['fp_soft']
fn = proposed[proposed['true'] < 20].sum()['fn_soft']
error = proposed[proposed['true'] < 20].mean()['error']
count = proposed[proposed['true'] < 20].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 20%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'].between(20, 40)].sum()['tp_soft']
fp = proposed[proposed['true'].between(20, 40)].sum()['fp_soft']
fn = proposed[proposed['true'].between(20, 40)].sum()['fn_soft']

error = proposed[proposed['true'].between(20, 40)].mean()['error']
count = proposed[proposed['true'].between(20, 40)].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 40%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'].between(40, 60)].sum()['tp_soft']
fp = proposed[proposed['true'].between(40, 60)].sum()['fp_soft']
fn = proposed[proposed['true'].between(40, 60)].sum()['fn_soft']

error = proposed[proposed['true'].between(40, 60)].mean()['error']
count = proposed[proposed['true'].between(40, 60)].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("40 - 60 %: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'] >= 60].sum()['tp_soft']
fp = proposed[proposed['true'] >= 60].sum()['fp_soft']
fn = proposed[proposed['true'] >= 60].sum()['fn_soft']
error = proposed[proposed['true'] >= 60].mean()['error']
count = proposed[proposed['true'] >= 60].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Greater than 60%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['clouds'] < 0.75].sum()['tp_soft']
fp = proposed[proposed['clouds'] < 0.75].sum()['fp_soft']
fn = proposed[proposed['clouds'] < 0.75].sum()['fn_soft']
error = proposed[proposed['clouds'] < 0.75].mean()['error']
count = proposed[proposed['clouds'] < 0.75].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 0.75 cloud: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['clouds'] >= 0.75].sum()['tp_soft']
fp = proposed[proposed['clouds'] >= 0.75].sum()['fp_soft']
fn = proposed[proposed['clouds'] >= 0.75].sum()['fn_soft']
error = proposed[proposed['clouds'] >= 0.75].mean()['error']
count = proposed[proposed['clouds'] >= 0.75].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Above 0.75 cloud: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['slopes'] < 0.1].sum()['tp_soft']
fp = proposed[proposed['slopes'] < 0.1].sum()['fp_soft']
fn = proposed[proposed['slopes'] < 0.1].sum()['fn_soft']
error = proposed[proposed['slopes'] < 0.1].mean()['error']
count = proposed[proposed['slopes'] < 0.1].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 0.1 slope: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['slopes'] >= 0.1].sum()['tp_soft']
fp = proposed[proposed['slopes'] >= 0.1].sum()['fp_soft']
fn = proposed[proposed['slopes'] >= 0.1].sum()['fn_soft']
error = proposed[proposed['slopes'] >= 0.1].mean()['error']
count = proposed[proposed['slopes'] >= 0.1].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Above 0.1 slope: ", precision, recall, error, count)

In [ ]:
proposed = pd.read_csv("../../data/metrics/rf.csv")
print(proposed.mean()['soft_prec'], proposed.mean()['soft_rec'])

# Load U-Net forest predictions

In [ ]:
proposed = pd.read_csv("../../data/metrics/unet-sample.csv")
proposed = proposed.drop(['Unnamed: 0'], axis = 1)
proposed = proposed.join(continents, how = 'inner')
proposed = proposed.drop(['lats'], axis = 1)
proposed = proposed.drop(['longs'], axis = 1)
proposed.head(5)

In [ ]:
proposed.groupby("aridity").count()

In [ ]:
proposed = proposed.merge(aridity, on = 'id')
sums = proposed.groupby("aridity").sum()
for i in proposed['aridity'].unique():
    tp = sums[sums.index == i]['tp_soft']
    fp = sums[sums.index == i]['fp_soft']
    fn = sums[sums.index == i]['fn_soft']
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print(i, float(precision), float(recall))

In [ ]:
means_l = []
means = []
means_u = []
proposed['ccgroup'] = 0
proposed['ccgroup'][(proposed['group'] >= 30) & (proposed['group'] <= 70)] = 1
for i in range(0, 100, 10):
    group = proposed[proposed['group'] == i]
    ci = mean_confidence_interval(group['error'], 0.95)
    if i >= 30 and i <= 70:
        means_l.append(ci[0])
        means_u.append(ci[2])
        means.append(ci[1])
        print(i, ci)
print(np.mean(means_l), np.mean(means), np.mean(means_u))

print(mean_confidence_interval(proposed[proposed['ccgroup'] == 1]['error']))

In [ ]:
mean_confidence_interval(proposed['error'], 0.95)

In [ ]:
sums = proposed.groupby("CONTINENT").sum()
for i in ['Africa', 'Asia', "Australia", "Europe", "North America", "South America"]:
    tp = sums[sums.index == i]['tp_soft']
    fp = sums[sums.index == i]['fp_soft']
    fn = sums[sums.index == i]['fn_soft']
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print(i, float(precision), float(recall))

In [ ]:
tp = proposed[proposed['true'] < 20].sum()['tp_soft']
fp = proposed[proposed['true'] < 20].sum()['fp_soft']
fn = proposed[proposed['true'] < 20].sum()['fn_soft']
error = proposed[proposed['true'] < 20].mean()['error']
count = proposed[proposed['true'] < 20].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 20%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'].between(20, 40)].sum()['tp_soft']
fp = proposed[proposed['true'].between(20, 40)].sum()['fp_soft']
fn = proposed[proposed['true'].between(20, 40)].sum()['fn_soft']

error = proposed[proposed['true'].between(20, 40)].mean()['error']
count = proposed[proposed['true'].between(20, 40)].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Between 20 and 40%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'].between(40, 60)].sum()['tp_soft']
fp = proposed[proposed['true'].between(40, 60)].sum()['fp_soft']
fn = proposed[proposed['true'].between(40, 60)].sum()['fn_soft']

error = proposed[proposed['true'].between(40, 60)].mean()['error']
count = proposed[proposed['true'].between(40, 60)].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("40 - 60 %: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'] >= 60].sum()['tp_soft']
fp = proposed[proposed['true'] >= 60].sum()['fp_soft']
fn = proposed[proposed['true'] >= 60].sum()['fn_soft']
error = proposed[proposed['true'] >= 60].mean()['error']
count = proposed[proposed['true'] >= 60].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Greater than 60%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['clouds'] >= 0.75].sum()['tp_soft']
fp = proposed[proposed['clouds'] >= 0.75].sum()['fp_soft']
fn = proposed[proposed['clouds'] >= 0.75].sum()['fn_soft']
error = proposed[proposed['clouds'] >= 0.75].mean()['error']
count = proposed[proposed['clouds'] >= 0.75].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Above 0.75 cloud: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['slopes'] >= 0.1].sum()['tp_soft']
fp = proposed[proposed['slopes'] >= 0.1].sum()['fp_soft']
fn = proposed[proposed['slopes'] >= 0.1].sum()['fn_soft']
error = proposed[proposed['slopes'] >= 0.1].mean()['error']
count = proposed[proposed['slopes'] >= 0.1].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Above 0.1 slope: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['slopes'] < 0.1].sum()['tp_soft']
fp = proposed[proposed['slopes'] < 0.1].sum()['fp_soft']
fn = proposed[proposed['slopes'] < 0.1].sum()['fn_soft']
error = proposed[proposed['slopes'] < 0.1].mean()['error']
count = proposed[proposed['slopes'] < 0.1].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 0.1 slope: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['clouds'] < 0.75].sum()['tp_soft']
fp = proposed[proposed['clouds'] < 0.75].sum()['fp_soft']
fn = proposed[proposed['clouds'] < 0.75].sum()['fn_soft']
error = proposed[proposed['clouds'] < 0.75].mean()['error']
count = proposed[proposed['clouds'] < 0.75].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 0.75 cloud: ", precision, recall, error, count)

In [ ]:
proposed = pd.read_csv("../../data/metrics/unet.csv")
print(proposed.mean()['soft_prec'], proposed.mean()['soft_rec'])

# Load SVM predictions

In [ ]:
proposed = pd.read_csv("../../data/metrics/SVM-sample.csv")
proposed = proposed.drop(['Unnamed: 0'], axis = 1)
proposed = proposed.drop(['lats'], axis = 1)
proposed = proposed.join(continents, how = 'inner')
proposed.head(5)



In [ ]:
proposed = proposed.join(aridity)
sums = proposed.groupby("aridity").sum()
for i in proposed['aridity'].unique():
    tp = sums[sums.index == i]['tp_soft']
    fp = sums[sums.index == i]['fp_soft']
    fn = sums[sums.index == i]['fn_soft']
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print(i, float(precision), float(recall))

In [ ]:
sums = proposed.groupby("CONTINENT").sum()
for i in ['Africa', 'Asia', "Australia", "Europe", "North America", "South America"]:
    tp = sums[sums.index == i]['tp_soft']
    fp = sums[sums.index == i]['fp_soft']
    fn = sums[sums.index == i]['fn_soft']
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print(i, float(precision), float(recall))

In [ ]:
tp = proposed[proposed['clouds'] < 0.75].sum()['tp_soft']
fp = proposed[proposed['clouds'] < 0.75].sum()['fp_soft']
fn = proposed[proposed['clouds'] < 0.75].sum()['fn_soft']
error = proposed[proposed['clouds'] < 0.75].mean()['error']
count = proposed[proposed['clouds'] < 0.75].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 0.75 cloud: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['clouds'] >= 0.75].sum()['tp_soft']
fp = proposed[proposed['clouds'] >= 0.75].sum()['fp_soft']
fn = proposed[proposed['clouds'] >= 0.75].sum()['fn_soft']
error = proposed[proposed['clouds'] >= 0.75].mean()['error']
count = proposed[proposed['clouds'] >= 0.75].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Above 0.75 cloud: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'] < 20].sum()['tp_soft']
fp = proposed[proposed['true'] < 20].sum()['fp_soft']
fn = proposed[proposed['true'] < 20].sum()['fn_soft']
error = proposed[proposed['true'] < 20].mean()['error']
count = proposed[proposed['true'] < 20].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 20%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'].between(20, 40)].sum()['tp_soft']
fp = proposed[proposed['true'].between(20, 40)].sum()['fp_soft']
fn = proposed[proposed['true'].between(20, 40)].sum()['fn_soft']

error = proposed[proposed['true'].between(20, 40)].mean()['error']
count = proposed[proposed['true'].between(20, 40)].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Between 20 and 40%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'].between(40, 60)].sum()['tp_soft']
fp = proposed[proposed['true'].between(40, 60)].sum()['fp_soft']
fn = proposed[proposed['true'].between(40, 60)].sum()['fn_soft']

error = proposed[proposed['true'].between(40, 60)].mean()['error']
count = proposed[proposed['true'].between(40, 60)].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Between 20 and 40%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['true'] >= 60].sum()['tp_soft']
fp = proposed[proposed['true'] >= 60].sum()['fp_soft']
fn = proposed[proposed['true'] >= 60].sum()['fn_soft']
error = proposed[proposed['true'] >= 60].mean()['error']
count = proposed[proposed['true'] >= 60].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Greater than 60%: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['slopes'] < 0.1].sum()['tp_soft']
fp = proposed[proposed['slopes'] < 0.1].sum()['fp_soft']
fn = proposed[proposed['slopes'] < 0.1].sum()['fn_soft']
error = proposed[proposed['slopes'] < 0.1].mean()['error']
count = proposed[proposed['slopes'] < 0.1].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Below 0.1 slope: ", precision, recall, error, count)

In [ ]:
tp = proposed[proposed['slopes'] >= 0.1].sum()['tp_soft']
fp = proposed[proposed['slopes'] >= 0.1].sum()['fp_soft']
fn = proposed[proposed['slopes'] >= 0.1].sum()['fn_soft']
error = proposed[proposed['slopes'] >= 0.1].mean()['error']
count = proposed[proposed['slopes'] >= 0.1].count()['error'] * 196



precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Above 0.1 slope: ", precision, recall, error, count)